##### The following code analyzes the viable_patches_json file. The points of analysis are described below. The primary tool for this analysis is pydriller.

1. Total size of the cloned repos
2. Total number of vulnerability inducing commits (vuln commits) found & not found
3. Average number of months between vuln commit and patch commit (or fix)
4. Average number of commits between the vuln commit & patch commit (or fix)
5. Average number of vuln commits fixed by patch commit (or fix)
6. Percentage of vulns where the vuln commit and fix were made by the same person


##### Sources
- @inbook{PyDriller,
    title = "PyDriller: Python Framework for Mining Software Repositories",
    abstract = "Software repositories contain historical and valuable information about the overall development of software systems. Mining software repositories (MSR) is nowadays considered one of the most interesting growing fields within software engineering. MSR focuses on extracting and analyzing data available in software repositories to uncover interesting, useful, and actionable information about the system. Even though MSR plays an important role in software engineering research, few tools have been created and made public to support developers in extracting information from Git repository. In this paper, we present PyDriller, a Python Framework that eases the process of mining Git. We compare our tool against the state-of-the-art Python Framework GitPython, demonstrating that PyDriller can achieve the same results with, on average, 50% less LOC and significantly lower complexity.URL: https://github.com/ishepard/pydrillerMaterials: https://doi.org/10.5281/zenodo.1327363Pre-print: https://doi.org/10.5281/zenodo.1327411",
    author = "Spadini, Davide and Aniche, Maurício and Bacchelli, Alberto",
    year = "2018",
    doi = "10.1145/3236024.3264598",
    booktitle = "The 26th ACM Joint European Software Engineering Conference and Symposium on the Foundations of Software Engineering (ESEC/FSE)",
    }

##### Author @Trust-Worthy



##### Reading in the results from the patch_vuln_match.json file and processing objects according to JSONL standard

In [1]:
import pandas as pd
import jsonlines    

json_path:str = "../production_ready/patch_vuln_match.jsonl"

data: list[object] = []

with jsonlines.open(json_path) as reader:

    data = [entry for entry in reader]

# Convert the list of dictionaries into a pandas DataFrame
patch_vuln_df = pd.DataFrame(data)


# Define a function to extract the file paths and commits
def extract_vuln_files_commits(vuln_commits):
    if vuln_commits:
        files = list(vuln_commits.keys())
        commits = [commit for commits in vuln_commits.values() for commit in commits]
        return pd.Series([files, commits])
    else:
        return pd.Series([[], []])  # Empty lists if no vuln_commits

# Apply the function to create new columns
patch_vuln_df[['vuln_files', 'vuln_commits']] = patch_vuln_df['vuln_commits'].apply(extract_vuln_files_commits)



print(patch_vuln_df.head())


           cve_id                     repo  \
0   CVE-1999-0199             bminor/glibc   
1   CVE-1999-0731         KDE/kde1-kdebase   
2   CVE-2002-2443                krb5/krb5   
3  CVE-2005-10002  wp-plugins/secure-files   
4  CVE-2005-10003      mikexstudios/xcomic   

                               patch_commit  \
0  2864e767053317538feafa815046fff89e5a16be   
1  04906bd5de2f220bf100b605dad37b4a1d9a91a6   
2  cf1a0c411b2668c57c41e9c4efd15ba17b6b322c   
3  cab025e5fc2bcdad8032d833ebc38e6bd2a13c92   
4  6ed8e3cc336e29f09c7e791863d0559939da98bf   

                                        vuln_commits  \
0  [dc5efe83c0252ad45337ab98eff6c26fdb29b0a9, 27a...   
1                                                 []   
2  [e88f857c3680ea395c0bed6a82862d8ea1177221, 438...   
3         [b1afc063fd49cfb875e1c6f591543ebff6649469]   
4                                                 []   

                                          vuln_files  
0  [elf/dl-load.c, manual/search.texi, misc/sysl

##### This is where the fun begins.... (iykyk)

In [ ]:




"""
Obtaining ...

1. Total size of the cloned repos
2. Total number of vulnerability inducing commits (vuln commits) found & (& num of patches missing a vuln (not found))
3. Average number of months between vuln commit and patch commit (or fix)
4. Average number of commits between the vuln commit & patch commit (or fix)
5. Average number of vuln commits fixed by patch commit (or fix)
6. Percentage of vulns where the vuln commit and fix were made by the same person
"""



import os
import shutil
from pydriller import Repository, Commit
from datetime import datetime
from dateutil.relativedelta import relativedelta
from git import Repo

# Calculate repo size
def get_directory_size(path: str) -> float:
    size: float = 0
    for dirpath, _, filenames in os.walk(path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            size += os.path.getsize(fp)
    return size


### Point 1
SIZE_OF_ALL_CLONED_REPOS: float = 0 ### size in MB

### Point 2
TOTAL_VULNS: int = 0 ### Another way to say this is total patch vuln pairs
TOTAL_PATCH_COMMITS_W_VULN_COMMIT: int = 0


### Point 6
### I can get the the number of patches without vulns / not found by doing total entires - total vulns
BY_SAME_PERSON: int = 0 ### Num of vulns made by the same person
PERCENTAGE_OF_VULN_N_PATCH_BY_SAME_PERSON: float = 0.0


### Point 3
TOTAL_NUM_MONTHS_BETWEEN: int = 0
AVERAGE_NUM_MONTHS_BETWEEN_VULN_N_PATCH: float = 0.0

### Point 4
TOTAL_NUM_COMMITS_BETWEEN: int = 0
AVERAGE_NUM_COMMITS_BETWEEN_VULN_N_PATCH: float = 0.0

### Variable used to track repos analyzed for Point 1 so that we get accurate storage metrics
unique_repo_paths: set[str] = set()
iteration: int = 0
### Point 1, 3, 4 , 6
for owner_repo, patch_commit, vuln_commits in zip(
    patch_vuln_df["repo"], 
    patch_vuln_df["patch_commit"], 
    patch_vuln_df["vuln_commits"]
):
    
    print(f"Working on iteration --{iteration}-- of df")

    print("Owner repo:" + owner_repo)
    print("Patch Commit:" + patch_commit)
    print("Vuln commits " + str(vuln_commits))

    ## If there aren't any commits to analyze, go onto the next iteration
    if vuln_commits == []:
        continue

    # Compose remote repo for pydriller
    owner, repo = owner_repo.split("/")
    remote_url: str = f"https://github.com/{owner}/{repo}.git"

    

    commits_to_analyze: list[str] = []
    commits_to_analyze.append(patch_commit)
    commits_to_analyze.extend(vuln_commits)

    print("commits to analyze: " + str(commits_to_analyze))

    ### Vars for point 3 and point 4
    temp_repo: Repository = Repository(remote_url, only_commits=commits_to_analyze, order='reverse')
    patch_author: str = ""
    patch_author_date: datetime = None
    patch_hash: str = ""
    is_patch: bool = True
    temp_repo_path: str = ""
    for commit in temp_repo.traverse_commits(): ### First commit will be 
        
       
        ### Code for point 1
        if commit.project_path not in unique_repo_paths:
            
            ### temp repo path updates every time there is a new path commit
            temp_repo_path = commit.project_path  
            unique_repo_paths.add(temp_repo_path)
            repo_size: float = get_directory_size(temp_repo_path) / (1024 * 1024)  # Convert to MB
            print("temp repo path:")
            print("uniqu repo paths" + str(unique_repo_paths))
            print("repo size:" + str(repo_size))
        

        if is_patch:
            patch_author_date = commit.author_date

            ### Point 6
            patch_author = commit.author.email ## is that the correct syntax? what type of object is being returned

            ### Point 4
            patch_hash = commit.hash
            ## this patch HAS at least one vuln commit
            TOTAL_PATCH_COMMITS_W_VULN_COMMIT += 1 
            is_patch = False
            continue ### This line is INCREDIBLY Important

        ### Code for point 3 & Point 6
        ############################################################
        vuln_author: str = ""
        vuln_author_date: datetime = None

        ### Point 3
        ### reassing the value of vuln_author_date on each iteration when is_patch is false
        vuln_author_date = commit.author_date

        ### Point 4
        vuln_hash = commit.hash

        ### Point 6
        vuln_author = commit.author.email
            
        ### Point 3
        ### Calculate difference between patch date and vuln date in months

        if patch_author_date is not None and vuln_author_date is not None:
            difference = relativedelta(patch_author_date, vuln_author_date)
            print(f"difference: {difference}")

            months_difference = (difference.years or 0) * 12 + (difference.months or 0)
            TOTAL_NUM_MONTHS_BETWEEN += months_difference
        else:
            print("Skipping calculation: Missing date values")



        

        ### Code for point 4
        temp_repo_obj: Repo = Repo(temp_repo_path)
        TOTAL_NUM_COMMITS_BETWEEN += int(temp_repo_obj.git.rev_list(f"{vuln_hash}..{patch_hash}", count=True))
        
        ### Point 6
        ### Compare patch author and vuln author
        if patch_author == vuln_author:
            BY_SAME_PERSON += 1
          
        #shutil.rmtree(temp_repo_path)

    ### Point 1
    SIZE_OF_ALL_CLONED_REPOS += repo_size

    

    ### Point 2
    TOTAL_VULNS += len(vuln_commits) ### Getting total vulns
    iteration +=1

    
    
    
    





Working on iteration --0-- of df
Owner repo:bminor/glibc
Patch Commit:2864e767053317538feafa815046fff89e5a16be
Vuln commits ['dc5efe83c0252ad45337ab98eff6c26fdb29b0a9', '27aa0631c73ee805519f3ac078eaef460f1b4bc3', '2604afb1b2d9acc3c70b1214285f996200bf0358', '50304ef0572fb41ba853262046dc7594f6a15241', '0324daa0055227fdb157b8491d4e5bbe9d9d579a', '28f540f45bbacd939bfd07f213bcad2bf730b1bf', 'c90a2db6e0c4661091a9c5b1454a9beffc611e9e', '2b83a2a4d978012cdf78b648337c31091e20526d', 'c4563d2d668675c2d5083f6ad01c801d6ccbe013', '50304ef0572fb41ba853262046dc7594f6a15241', 'ba1ffaa1c6989873b57edc84491bfd1308b2190d', 'ee188d555b8c32ad9704a7440cab400af967292f', 'f21acc89c06c14160eab88246e9dbe0b17eb5f89', '569c558c880779d33c6642662d1aa57dff697244', '5929563ff20245bbaa1b3ca2f4bfcee623ac6fb5', '2b79b6d54362c8b9833488b1c88f1259d8c16a55', '8a52392237c44cbbd1ffc62b164230e1159dfb76', '569c558c880779d33c6642662d1aa57dff697244', '5929563ff20245bbaa1b3ca2f4bfcee623ac6fb5', '2b79b6d54362c8b9833488b1c88f1259d8c16a

In [ ]:
"""
Calculating final values:
"""
### Point 2
TOTAL_ENTIRES = len(patch_vuln_df)
PATCHES_WO_VULN = TOTAL_ENTIRES - TOTAL_PATCH_COMMITS_W_VULN_COMMIT

### Point 3
AVERAGE_NUM_MONTHS_BETWEEN_VULN_N_PATCH: float = (TOTAL_NUM_MONTHS_BETWEEN / TOTAL_PATCH_COMMITS_W_VULN_COMMIT )

### Point 4
AVERAGE_NUM_COMMITS_BETWEEN_VULN_N_PATCH: float = (TOTAL_NUM_COMMITS_BETWEEN / TOTAL_PATCH_COMMITS_W_VULN_COMMIT)

### Point 5
AVERAGE_NUM_OF_VULNS_TO_PATCH: float = (TOTAL_VULNS / TOTAL_PATCH_COMMITS_W_VULN_COMMIT)


### Point 6
PERCENTAGE_OF_VULN_N_PATCH_BY_SAME_PERSON: float = (TOTAL_VULNS / BY_SAME_PERSON ) * 100